In [4]:
import pvporcupine
import pyaudio
import wave
import struct
import os
import sys

In [5]:
def record_audio_with_fixed_duration(path, filename, duration):
    # Set the parameters for the audio stream
    chunk = 1024
    sample_format = pyaudio.paInt16
    channels = 1
    fs = 44100
    
    # Initialize the PyAudio object
    p = pyaudio.PyAudio()
    
    # Open the audio stream
    stream = p.open(format=sample_format,
                    channels=channels,
                    rate=fs,
                    frames_per_buffer=chunk,
                    input=True)
    
    frames = []
    
    # Record the audio for the specified duration   
    for i in range(int(fs / chunk * duration)):
        data = stream.read(chunk)
        frames.append(data)
    
    # Stop and close the audio stream
    stream.stop_stream()
    stream.close()
    
    # Terminate the PyAudio object
    p.terminate()
    
    # Save the recorded audio as a WAV file
    file_path = os.path.join(path, filename)
    wf = wave.open(file_path, 'wb')
    wf.setnchannels(channels)
    wf.setsampwidth(p.get_sample_size(sample_format))
    wf.setframerate(fs)
    wf.writeframes(b''.join(frames))
    wf.close()

In [6]:
# https://console.picovoice.ai/
pico_key = os.environ["PICOVOICE_API_KEY"]
path="/home/sougato97/Human_Robot_Interaction/nao_dev/live_audio/hey_aiko_ja_linux_v2_1_0.ppn"
# https://github.com/Picovoice/porcupine/tree/master/lib/common

porcupine = pvporcupine.create(access_key=pico_key, keyword_paths=[path], model_path="/home/sougato97/Human_Robot_Interaction/nao_dev/live_audio/porcupine_params_ja.pv")

# Initialize PyAudio and open a stream
pa = pyaudio.PyAudio()
audio_stream = pa.open(
    rate=porcupine.sample_rate,
    channels=1,
    format=pyaudio.paInt16,
    input=True,
    frames_per_buffer=porcupine.frame_length)

# Create a wave file to store the recording
wf = wave.open("recording.wav", "wb")
wf.setnchannels(1)
wf.setsampwidth(pa.get_sample_size(pyaudio.paInt16))
wf.setframerate(porcupine.sample_rate)

# Loop until the wake word is detected
print("Listening for wake word...")
while True:
    # Read a frame of audio
    pcm = audio_stream.read(porcupine.frame_length)
    pcm = struct.unpack_from("h" * porcupine.frame_length, pcm)

    # Process the frame with Porcupine
    keyword_index = porcupine.process(pcm)

    # If the wake word is detected, break the loop
    if keyword_index >= 0:
        print("Wake word detected!")
        break

# Loop until the user stops speaking
# print("Recording...")
# while True:
#     # Read a frame of audio
#     pcm = audio_stream.read(porcupine.frame_length)
#     pcm = struct.unpack_from("h" * porcupine.frame_length, pcm)

#     # Write the frame to the wave file
#     wf.writeframes(struct.pack("h" * len(pcm), *pcm))

#     # Check if the user has stopped speaking by measuring the energy of the frame
#     energy = sum([abs(x) for x in pcm]) / len(pcm)
#     if energy < 100:
#         print("Recording stopped.")
#         break

# # Close and terminate everything
# wf.close()
# audio_stream.close()
# pa.terminate()

record_audio_with_fixed_duration("/home/sougato97/Human_Robot_Interaction/nao_dev/live_audio", "test_recording.mp3", 7)

porcupine.delete()

KeyboardInterrupt: 